## Creating Positive and Negative Datasets

In [ ]:
import cv2
import numpy as np
import os

## For negative images, replace pos by neg

def store_raw_images():
    if not os.path.exists('pos'):
        os.makedirs('pos')
    pic_num = 1
    for img in os.listdir('pos'):
        try:
            print(img)
            img = cv2.imread('pos/' + str(img), cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(img, (100, 100))
            cv2.imwrite('pos_2/' + str(pic_num) + '_1_0_0_50_50.jpg', resized_image)
            pic_num += 1
        except Exception as e:
            print(str(e))

## To remove any empty files from the dataset

def find_uglies():
    for file_type in ['neg']:
        for img in os.listdir(file_type):
            for ugly in os.listdir('uglies'):
                try:
                    current_image_path = str(file_type) + '/' + str(img)
                    ugly = cv2.imread('uglies/' + str(ugly))
                    question = cv2.imread(current_image_path)

                    if ugly.shape ==  question.shape and not(np.bitwise_xor(ugly, question).any()):
                        print('Ugly Image')
                        print(current_image_path)
                        os.remove(current_image_path)
                except Exception as e:
                    print(str(e))

def create_pos_n_neg():
    for file_type in ['pos']:
        for img in os.listdir(file_type):
            if file_type == 'neg':
                line = file_type + '/' + img + '\n'
                with open('bg.txt', 'a') as f:
                    f.write(line)
            elif file_type == 'pos':
                line = img + ' 1 0 0 50 50\n'
                with open('info.lst', 'a') as f:
                    f.write(line)
store_raw_images()
#find_uglies()
#create_pos_n_neg()

## Using the watch-cascade.xml file to predict

In [ ]:
import numpy as np
import cv2

# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
watch_cascade = cv2.CascadeClassifier('watch-cascade.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    watches = watch_cascade.detectMultiScale(gray, 1200, 1200)
    
    for (x, y, w, h) in watches:
        font = cv2.FONT_HERSHEY_SIMPLEX
        #cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)
        cv2.putText(image, 'Watch', (x, y), font, 0.5, (255, 255, 0), 2, cv2.LINE_AA)

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
        #cv2.putText(image, 'Anjani', (x, y), font, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = image[y:y + h, x:x + w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, eh, ew) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            cv2.putText(roi_color, 'Eyes', (ex, ey), font, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('image', image)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()